## Exercise 5

In [1]:
using LinearAlgebra, MLBase, GLMNet, DataFrames, CSV, Random, Plots, DecisionTree, RDatasets, PrettyTables, DataStructures, NearestNeighbors, Distances, LIBSVM, Distances

In [2]:
categorize_value(pred_val) = argmin(abs.(pred_val .- 1))
calculate_accuracy(predicted_vals, actual_vals) = sum(predicted_vals .== actual_vals) / length(actual_vals)

calculate_accuracy (generic function with 1 method)

In [3]:
function split_by_year(data_years, proportion)
    unique_years = unique(data_years)
    selected_ids = []
    for year in unique_years
        indices = findall(data_years .== year)
        sampled_ids = randsubseq(indices, proportion)
        append!(selected_ids, sampled_ids...)
    end
    return selected_ids
end

split_by_year (generic function with 1 method)

In [4]:
stock_data = CSV.read("/Users/michelletorres/Desktop/Homeworks AI/Smarket.csv", DataFrame)
@show(stock_data)
@show(size(stock_data))

stock_data = 1250×9 DataFrame
  Row │ Year   Lag1     Lag2     Lag3     Lag4     Lag5     Volume   Today    Direction
      │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  String7
──────┼─────────────────────────────────────────────────────────────────────────────────
    1 │  2001    0.381   -0.192   -2.624   -1.055    5.01   1.1913     0.959  Up
    2 │  2001    0.959    0.381   -0.192   -2.624   -1.055  1.2965     1.032  Up
    3 │  2001    1.032    0.959    0.381   -0.192   -2.624  1.4112    -0.623  Down
    4 │  2001   -0.623    1.032    0.959    0.381   -0.192  1.276      0.614  Up
    5 │  2001    0.614   -0.623    1.032    0.959    0.381  1.2057     0.213  Up
    6 │  2001    0.213    0.614   -0.623    1.032    0.959  1.3491     1.392  Up
    7 │  2001    1.392    0.213    0.614   -0.623    1.032  1.445     -0.403  Down
    8 │  2001   -0.403    1.392    0.213    0.614   -0.623  1.4078     0.027  Up
    9 │  2001    0.027   -0.403    1.392    0.213    0.

(1250, 9)

In [5]:
feature_matrix = Matrix(stock_data[:, 3:8])
println(names(stock_data))
labels = stock_data[:, 9]

["Year", "Lag1", "Lag2", "Lag3", "Lag4", "Lag5", "Volume", "Today", "Direction"]


1250-element PooledArrays.PooledVector{String7, UInt32, Vector{UInt32}}:
 "Up"
 "Up"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Down"
 "Up"
 ⋮
 "Up"
 "Down"
 "Down"
 "Down"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Up"
 "Down"
 "Down"

In [6]:
label_mapping = MLBase.labelmap(labels)
encoded_labels = labelencode(label_mapping, labels)

1250-element Vector{Int64}:
 1
 1
 2
 1
 1
 1
 2
 1
 1
 1
 2
 2
 1
 ⋮
 1
 2
 2
 2
 2
 1
 1
 1
 2
 1
 2
 2

In [7]:
training_indices = split_by_year(feature_matrix[:, 2], 0.7)
@show size(training_indices)
testing_indices = setdiff(1:length(feature_matrix[:, 2]), training_indices)
@show size(testing_indices)

size(training_indices) = (847,)
size(testing_indices) = (403,)


(403,)

## Lasso Regression

In [8]:
lasso_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices])
lasso_cv = glmnetcv(feature_matrix[training_indices, :], encoded_labels[training_indices])
optimal_lambda_lasso = lasso_model.lambda[argmin(lasso_cv.meanloss)]
lasso_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices], lambda=[optimal_lambda_lasso])
lasso_predictions = GLMNet.predict(lasso_model, feature_matrix[testing_indices, :])
lasso_predictions = categorize_value.(lasso_predictions)
calculate_accuracy(lasso_predictions, encoded_labels[testing_indices])

0.5086848635235732

## Ridge Regression

In [9]:
ridge_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0)
ridge_cv = glmnetcv(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0)
optimal_lambda_ridge = ridge_model.lambda[argmin(ridge_cv.meanloss)]
ridge_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0, lambda=[optimal_lambda_ridge])
ridge_predictions = GLMNet.predict(ridge_model, feature_matrix[testing_indices, :])
ridge_predictions = categorize_value.(ridge_predictions)
calculate_accuracy(ridge_predictions, encoded_labels[testing_indices])

0.5086848635235732

## Elastic Net

In [10]:
elastic_net_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0.5)
elastic_net_cv = glmnetcv(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0.5)
optimal_lambda_en = elastic_net_model.lambda[argmin(elastic_net_cv.meanloss)]
elastic_net_model = glmnet(feature_matrix[training_indices, :], encoded_labels[training_indices], alpha=0.5, lambda=[optimal_lambda_en])
en_predictions = GLMNet.predict(elastic_net_model, feature_matrix[testing_indices, :])
en_predictions = categorize_value.(en_predictions)
calculate_accuracy(en_predictions, encoded_labels[testing_indices])

0.5086848635235732

## Decision Tree

In [11]:
tree_model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(tree_model, feature_matrix[training_indices, :], encoded_labels[training_indices])
tree_predictions = DecisionTree.predict(tree_model, feature_matrix[testing_indices, :])
calculate_accuracy(tree_predictions, encoded_labels[testing_indices])

1.0

## Random Forest

In [13]:
rf_model = DecisionTree.RandomForestClassifier(n_trees=20)
DecisionTree.fit!(rf_model, feature_matrix[training_indices, :], encoded_labels[training_indices])
rf_predictions = DecisionTree.predict(rf_model, feature_matrix[testing_indices, :])
calculate_accuracy(rf_predictions, encoded_labels[testing_indices])

1.0

## k-Nearest Neighbors

In [14]:
train_features = feature_matrix[training_indices, :]
train_labels = encoded_labels[training_indices]
kdtree_model = KDTree(train_features')
query_points = feature_matrix[testing_indices, :]
nearest_indices, distances = knn(kdtree_model, query_points', 5, true)
neighbors_labels = train_labels[hcat(nearest_indices...)]
label_counts = map(i -> counter(neighbors_labels[:, i]), 1:size(neighbors_labels, 2))
knn_predictions = map(i -> parse(Int, string(argmax(label_counts[i]))), 1:size(neighbors_labels, 2))
calculate_accuracy(knn_predictions, encoded_labels[testing_indices])

0.9181141439205955

In [15]:
svm_model = svmtrain(train_features', train_labels) # Support vector machine
svm_predictions, decision_values = svmpredict(svm_model, feature_matrix[testing_indices, :]')
calculate_accuracy(svm_predictions, encoded_labels[testing_indices])

0.9751861042183623

In [16]:
accuracy_scores = zeros(7) # Compare overall accuracies
methods_list = ["Lasso", "Ridge", "ElasticNet", "DecisionTree", "RandomForest", "kNN", "SVM"]
test_labels = encoded_labels[testing_indices]
accuracy_scores[1] = calculate_accuracy(lasso_predictions, test_labels)
accuracy_scores[2] = calculate_accuracy(ridge_predictions, test_labels)
accuracy_scores[3] = calculate_accuracy(en_predictions, test_labels)
accuracy_scores[4] = calculate_accuracy(tree_predictions, test_labels)
accuracy_scores[5] = calculate_accuracy(rf_predictions, test_labels)
accuracy_scores[6] = calculate_accuracy(knn_predictions, test_labels)
accuracy_scores[7] = calculate_accuracy(svm_predictions, test_labels)
hcat(methods_list, accuracy_scores)

7×2 Matrix{Any}:
 "Lasso"         0.508685
 "Ridge"         0.508685
 "ElasticNet"    0.508685
 "DecisionTree"  1.0
 "RandomForest"  1.0
 "kNN"           0.918114
 "SVM"           0.975186

## Confusion matrices

In [18]:
println("Confusion matrix Lasso")
pretty_table(confusmat(2, test_labels, lasso_predictions[:]))
println("Confusion matrix Ridge")
pretty_table(confusmat(2, test_labels, ridge_predictions[:]))
println("Confusion matrix Elastic Net")
pretty_table(confusmat(2, test_labels, en_predictions[:]))
println("Confusion matrix Decision Tree")
pretty_table(confusmat(2, test_labels, tree_predictions[:]))
println("Confusion matrix Random Forest")
pretty_table(confusmat(2, test_labels, rf_predictions[:]))
println("Confusion matrix kNN")
pretty_table(confusmat(2, test_labels, knn_predictions[:]))
println("Confusion matrix SVM")
pretty_table(confusmat(2, test_labels, svm_predictions[:]))

Confusion matrix Lasso
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    205 │      0 │
│    198 │      0 │
└────────┴────────┘
Confusion matrix Ridge
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    205 │      0 │
│    198 │      0 │
└────────┴────────┘
Confusion matrix Elastic Net
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    205 │      0 │
│    198 │      0 │
└────────┴────────┘
Confusion matrix Decision Tree
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    205 │      0 │
│      0 │    198 │
└────────┴────────┘
Confusion matrix Random Forest
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    205 │      0 │
│      0 │    198 │
└────────┴────────┘
Confusion matrix kNN
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    198 │      7 │
│     26 │    172 │
└────────┴────────┘
Confusion matrix SVM
┌────────┬────────┐
│ Col. 1 │ Col. 2 │
├────────┼────────┤
│    204 │      1 │
│      9 │    189 │
└